In [1]:
import os, cPickle
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from scipy.io import loadmat

from sklearn.decomposition import PCA

from palettable.colorbrewer.qualitative import Set1_9
colors = Set1_9.mpl_colors
cmap = Set1_9.mpl_colormap

%matplotlib inline

In [2]:
data_dir = "whole brain data Kato et al 2015"
zimmer_state_labels = \
    loadmat(os.path.join(
            data_dir, 
            "sevenStateColoring.mat"))
    
worm_files = ["TS20140715e_lite-1_punc-31_NLS3_2eggs_56um_1mMTet_basal_1080s.mat",
              "TS20140715f_lite-1_punc-31_NLS3_3eggs_56um_1mMTet_basal_1080s.mat",
              "TS20140905c_lite-1_punc-31_NLS3_AVHJ_0eggs_1mMTet_basal_1080s.mat",
              "TS20140926d_lite-1_punc-31_NLS3_RIV_2eggs_1mMTet_basal_1080s.mat",
              "TS20141221b_THK178_lite-1_punc-31_NLS3_6eggs_1mMTet_basal_1080s.mat"
             ]


In [40]:
all_first_neuron_ids = []
all_second_neuron_ids = []
all_neuron_ids = []
for worm_index in range(5):
    zimmer_data = loadmat(os.path.join(data_dir, "wbdata", worm_files[worm_index]))
    
    # Get the state sequence as labeled in Kato et al
    zimmer_key = map(lambda x: str(x[0]), zimmer_state_labels["sevenStateColoring"]["key"][0,0][0])
    zimmer_states = zimmer_state_labels["sevenStateColoring"]["dataset"][0,0]['stateTimeSeries'][0,worm_index].ravel() - 1
    zimmer_cps = np.concatenate(([0], 1+np.where(np.diff(zimmer_states))[0], [zimmer_states.size-1]))
    N_zimmer = zimmer_states.max() + 1

    # Get the calcium trace (corrected for bleaching)
    tt = zimmer_data["wbData"]['tv'][0,0]
    dff = zimmer_data["wbData"]['deltaFOverF'][0,0]
    dff_bc = zimmer_data["wbData"]['deltaFOverF_bc'][0,0]
    dff_deriv = zimmer_data["wbData"]['deltaFOverF_deriv'][0,0]

    neuron_ids_array = zimmer_data["wbData"]['NeuronIds'][0,0][0]
    first_neuron_ids_array = np.array(map(lambda x: None if len(x[0])==0 
                                          else str(x[0][0][0]) , 
                                          neuron_ids_array))
    second_neuron_ids_array = np.array(map(lambda x: None if x.size<2 or x[0,1].size == 0 
                                          else str(x[0,1][0]) , 
                                          neuron_ids_array))
    
    # Store
    all_first_neuron_ids.append(first_neuron_ids_array)
    all_second_neuron_ids.append(second_neuron_ids_array)
    all_neuron_ids.append(np.column_stack((first_neuron_ids_array, 
                                           second_neuron_ids_array)))


In [47]:
from functools import reduce
common_neurons = reduce(np.intersect1d, all_first_neuron_ids)
print(common_neurons)
common_neurons.shape
# map(np.unique, all_first_neuron_ids)

[None 'AIBL' 'AIBR' 'ALA' 'AVAL' 'AVAR' 'AVBL' 'AVBR' 'AVER' 'RIBL' 'RID'
 'RIML' 'RIMR' 'RIVL' 'RIVR' 'RMED' 'RMEL' 'RMER' 'SMDVR' 'VB01' 'VB02']


(21,)

In [30]:
np.array(map(lambda x: x.size, neuron_ids_array))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0])